In [1]:
from tensorflow.keras import layers, models
from tensorflow.keras.utils import to_categorical
import torch
from torch import nn, optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
data = pd.read_csv('winequality-red.csv')
data

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
1,7.8,0.880,0.00,2.6,0.098,25.0,67.0,0.99680,3.20,0.68,9.8,5
2,7.8,0.760,0.04,2.3,0.092,15.0,54.0,0.99700,3.26,0.65,9.8,5
3,11.2,0.280,0.56,1.9,0.075,17.0,60.0,0.99800,3.16,0.58,9.8,6
4,7.4,0.700,0.00,1.9,0.076,11.0,34.0,0.99780,3.51,0.56,9.4,5
...,...,...,...,...,...,...,...,...,...,...,...,...
1594,6.2,0.600,0.08,2.0,0.090,32.0,44.0,0.99490,3.45,0.58,10.5,5
1595,5.9,0.550,0.10,2.2,0.062,39.0,51.0,0.99512,3.52,0.76,11.2,6
1596,6.3,0.510,0.13,2.3,0.076,29.0,40.0,0.99574,3.42,0.75,11.0,6
1597,5.9,0.645,0.12,2.0,0.075,32.0,44.0,0.99547,3.57,0.71,10.2,5


In [3]:
y = data['quality'].values
X = data.drop(['quality'], axis=1).values


## Preparing the data

In [4]:
mean = X.mean(axis=0)
X -= mean
std = X.std(axis=0)
X /= std

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2, random_state=42)

## Building Dense network from task 3 on Keras

In [6]:
model1 = models.Sequential()
model1.add(layers.Dense(5, activation='relu', input_shape=(11,)))
model1.add(layers.Dense(1, activation='relu'))
model1.compile(optimizer='adam', loss='mse')

In [10]:
model1.fit(X_train, y_train, epochs=1000, batch_size=250)

In [8]:
y_test = np.resize(y_test, (320, 1))

In [9]:
model1_acc = sum(np.round(model1.predict(X_test)) == y_test) / 320 # it's better then my model in task 3, adam is cool
model1_acc

array([0.584375])

## Building Dense network on Keras

In [11]:
# some preparing data
to_class_y_train = to_categorical(y_train)
to_class_y_test = to_categorical(y_test)

In [12]:
model2 = models.Sequential()
model2.add(layers.Dense(256, activation='relu', input_shape=(11,)))
model2.add(layers.Dropout(0.2))
model2.add(layers.Dense(128, activation='relu'))
model2.add(layers.Dropout(0.2))
model2.add(layers.Dense(9, activation='softmax'))
model2.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

In [16]:
model2.fit(X_train, to_class_y_train, epochs=500, batch_size=250)

In [14]:
model2_acc = sum([[x.argmax()] for x in model2.predict(X_test)] == y_test) / 320
model2_acc


array([0.578125])

## Building Dense network on Pytorch

In [17]:
class model(nn.Module):
    def __init__(self):
        super(model, self).__init__()
        
        self.main = nn.Sequential(
            nn.Linear(11, 256),
            nn.LeakyReLU(),
            nn.Dropout(.2),
            
            nn.Linear(256, 128),
            nn.LeakyReLU(),
            nn.Dropout(.2),
            
            nn.Linear(128, 9),
            nn.Softmax(dim=1)
        )
    
    def forward(self, wine):
        wine = torch.from_numpy(wine).float()
        predict = self.main(wine)
        return predict

In [18]:
model3 = model()

In [19]:
loss_func = nn.CrossEntropyLoss()
optimizer = optim.SGD(model3.parameters(), lr=2e-3)

In [42]:
for epoch in range(4000):

    model3.train()

    i, n = 0, 1

    while X[i: n * 250].size != 0:

        inp = X_train[i:n * 250]
        y = torch.from_numpy(y_train[i:n * 250])
        n += 1
        i = n * 250

        optimizer.zero_grad()

        outputs = model3(inp)

        loss = loss_func(outputs, y)

        loss.backward()

        optimizer.step()

    if not epoch % 500:
        print("Epoch: {}/{}".format(epoch+1, 1000))
        print("Training: Loss: {:.4f}".format(loss.item()))
        print(sum([[x.argmax()] for x in model3(X_test)] == y_test) / 320)


In [41]:
model3.eval()
model3_acc = sum([[x.argmax()] for x in model3(X_test)] == y_test) / 320
model3_acc


array([0.559375])

## Building myself Dense Network and result evaluation

In [82]:
y_train = np.resize(y_train, (1279, 1))
y_test = np.resize(y_test, (320, 1))

In [83]:
layer_1 = np.random.random((11, 5)) * 2 - 1
layer_2 = np.random.random((5, 1)) * 2 - 1

In [84]:
def leaky_relu(layer, alpha=0.1):
    layer[layer < 0] *= alpha
    return layer

In [85]:
def predict(x):
    x = leaky_relu(x @ layer_1)
    x = leaky_relu(x @ layer_2)
    return x

In [105]:
def fit(epoch=4000, X=X_train, Y=y_train, batch_size=250, lr = 1e-5, layer_2=layer_2, layer_1=layer_1):
    for _ in range(epoch):

        i, n = 0, 1

        while X[i: n * batch_size].size != 0:

            inp = X[i:n * 250]
            y = np.resize(Y[i:n * 250], (inp.shape[0], 1))
            i = n * 250
            n += 1

            x_1 = leaky_relu(inp @ layer_1)
            out = leaky_relu(x_1 @ layer_2)

            error_grad = 2 * (y - out)

            layer_1 += (inp.T @ ((error_grad @ layer_2.T) * x_1)) * lr
            layer_2 += (x_1.T @ error_grad) * lr

In [115]:
fit(50000)


In [118]:
my_self_model_acc = sum(np.round(predict(X_test)) == y_test) / len(y_test)
my_self_model_acc

array([0.246875])

In [124]:
np.array([model1_acc - my_self_model_acc,
          model2_acc - my_self_model_acc,
          model3_acc - my_self_model_acc]).mean()

0.32708333333333334

my model on 33% worse them Keras and Pytorch
